# 모델 준비 part 클래스화 하기

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

## 아래 코드는 클래스로 작성된 코드와 동일하다.

In [7]:
# 모델을 준비합니다
X = tf.keras.Input(shape=[1])
Y = tf.keras.layers.Dense(1)(X)
model = tf.keras.Model(X, Y) 

In [ ]:
# 위 코드와 동일하다
# 모델을 준비합니다
class MyModel(tf.keras.Model): # tensorflow 모델 상속 받아옴
    def __init__(self, **kwargs): # 초기화 함수
        super(MyModel, self).__init__(**kwargs)
        self.dense = tf.keras.layers.Dense(1) # Y레이어 설정
        
    def call(self, input):# 여기서 input은 X를 받는다.
        Y = self.dense(input) # 여기서 input은 X를 받는다.
        return Y

In [4]:
model = MyModel()
model.compile(loss='mse')

## 히든레이어 2개짜리를 만든다고 할 때,

In [10]:
# 모델을 준비합니다
X = tf.keras.Input(shape=[1])
H = tf.keras.layers.Dense(2, activation='swish')(X)
Y = tf.keras.layers.Dense(1)(H)
model = tf.keras.Model(X, Y) 

In [11]:
# 위 코드와 동일하다
# 모델을 준비합니다
class MyModel(tf.keras.Model): # tensorflow 모델 상속 받아옴
    def __init__(self, **kwargs): # 초기화 함수
        super(MyModel, self).__init__(**kwargs)
        self.dense1 = tf.keras.layers.Dense(2, activation='swish') # H 레이어 설정
        self.dense2 = tf.keras.layers.Dense(1) # Y 레이어 설정
        
    def call(self, input):# 여기서 input은 X를 받는다.
        H = self.dense1(input) # 여기서 input은 X를 받는다.
        Y = self.dense2(H)
        return Y

In [ ]:
model = MyModel()
model.compile(loss='mse')